In [35]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import string


from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, FloatType
from __future__ import print_function, unicode_literals, with_statement, absolute_import, generators, nested_scopes, division
from operator import add


%matplotlib inline 

In [2]:
# Local Spark
# spark session is now the entry point of spark program
# for line 8, can use local[n] for run spark locally with n cores
spark = SparkSession.builder \
    .master('local') \
    .appName('Spark Try') \
    .getOrCreate()

In [3]:
data = spark.read.json('/project/cmsc25025/sou/speeches.json')
data.show()

+--------------------+--------------------+----+
|           president|                text|year|
+--------------------+--------------------+----+
|        James Monroe| Fellow-Citizens ...|1821|
|    William McKinley| To the Senate an...|1897|
|Dwight D. Eisenhower|[Delivered in per...|1960|
|     Calvin Coolidge|Since the close o...|1923|
|       James Madison| Fellow-Citizens ...|1816|
|    Grover Cleveland| To the Congress ...|1886|
|   John Quincy Adams| Fellow Citizens ...|1827|
|  Theodore Roosevelt| To the Senate an...|1905|
|   Lyndon B. Johnson|Mr. Speaker, Mr. ...|1965|
|       James K. Polk| Fellow-Citizens ...|1848|
|      Woodrow Wilson|Gentlemen of the ...|1913|
|Dwight D. Eisenhower|[Delivered in per...|1955|
|         George Bush|Mr. President and...|1991|
|     Franklin Pierce| Fellow-Citizens ...|1856|
...|1944|
|   Lyndon B. Johnson|[Delivered in per...|1968|
|      Andrew Johnson| Fellow-Citizens ...|1867|
|      Woodrow Wilson|GENTLEMEN OF THE ...|1914|
|      Har

# Part a
Compute the TF-IDF vectors for each SOU address. You should lower case all of the text, and remove punctuation.

For example, you could use something like this:
       s = s.lower().translate(string.maketrans("",""), string.punctuation)
You will have to make choices about the size of the term vocabulary to use—for example throwing out the 20 most common words, and words that appear fewer than, say, 50 times.

In [4]:
def trans(t):
    return t.lower().encode('utf-8').translate(string.maketrans("",""), string.punctuation).strip().split()

transRDD = data.rdd.map(lambda x:(int(x['year']), trans(x['text'])) )
num_doc = len(transRDD.collect())

In [5]:
allwords = transRDD.flatMap(lambda x: [(w,1) for w in x[1]]).reduceByKey(add).map(lambda x: (x[1], x[0]))\
            .sortByKey().filter(lambda x: int(x[0])>50)
vocab = allwords.map(lambda x: x[1]).collect()[:-20]

In [27]:
def df(l):
    res = []
    for word in vocab:
        if word in l:
            res.append((word, 1))
        else:
            res.append((word, 0))
    return res

def tf_and_df(x):
    y = x[0]
    l = x[1]
    res = []
    for word in vocab:
        c = l.count(word)
        doc_f = dfdic[word]
        tfidf = c*np.log(num_doc/doc_f)
        res.append([y,word, tfidf])
    return res
    

In [12]:
#get df
dflist = transRDD.flatMap(lambda x: df(x[1])).reduceByKey(add).collect()#.take(4)
#.takeOrdered(10, key=lambda x: -x)
dfdic = dict(dflist)

In [13]:
dfdic['intent']

37

In [29]:
row = transRDD.take(1)[0]
tf_and_df(row)[:5]

[[1821, 'intent', 0.0],
 [1821, 'enlarge', 0.0],
 [1821, 'ceded', 10.623490666339784],
 [1821, 'postoffices', 0.0],
 [1821, 'colleges', 0.0]]

In [31]:
transRDD.flatMap(lambda x: tf_and_df(x)).toDF()#.show(4)

TypeError: not supported type: <type 'numpy.float64'>

In [ ]:
#For each doc, get word frequency for words in vocabulary
#transRDD.flatMap(lambda x: tf_and_df(x)).reduceByKey(lambda x: add(x[-1])).take(10)
dfRDD.take(10)

###### vocab[:4]

In [ ]:
# l = transRDD.take(1)[0][1]
# word_freq_per_doc(l)

# Part b
Based on cosine similarity, find
- 50 most similar pairs of SOUs given by different Presidents.
- 50 most similar pairs of SOUs given by the same President.
- 25 most similar pairs of Presidents, averaging the cosine similarity over all pairsof their SOUs.

When you read the above speeches, do they indeed seem similar to you? (You can read the speeches in a more reader-friendly format here: http://www.presidency. ucsb.edu/sou.php) Comment on what you find, and describe what is needed to construct a better similarity measure between documents.

Interpretation:

# Part c
Using this vector representation, cluster the speeches using k-means.

Experiment with different number of clusters, and display the clusters obtained (in some manner that you choose). Comment on the clustering results, and whether or not the results are interpretable.

Although the SOU dataset is not very large, you should try to exploit parallelism whenever possible in order to become familiar with this paradigm.